Field() and StringConstraints()

In [1]:
from typing import Annotated
from pydantic import BaseModel, Field, StringConstraints, StrictStr, EmailStr

In [2]:
!pip install pydantic[email]

In [3]:
NonEmptyList = Annotated[list[StrictStr], Field(..., min_length=1), StringConstraints(min_length=3)]
StrippedString = Annotated[StrictStr, StringConstraints(strip_whitespace=True)]

In [4]:
class ValidationTst(BaseModel):
    variable: NonEmptyList = None
    var: StrippedString = Field(repr=False)

In [5]:
model = ValidationTst(var="      ")
model.var

''

In [6]:
LowerAlphaNumericStr = Annotated[StrictStr, StringConstraints(pattern=r"^[a-z0-9_]+$")]
SlackNameField = Annotated[str, StringConstraints(pattern=r"^[#@][a-z0-9_]+$")]

In [7]:
class ValidationTst2(BaseModel):
    name: SlackNameField

model = ValidationTst2(name="#")

ValidationError: 1 validation error for ValidationTst2
name
  String should match pattern '^[#@][a-z0-9_]+$' [type=string_pattern_mismatch, input_value='#', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/string_pattern_mismatch

In [11]:
Str2 = Annotated[StrictStr, StringConstraints(max_length=2)]
LowerEmailStr = Annotated[Str2, EmailStr, StringConstraints(to_lower=True, max_length=2)]

In [12]:
class Hello(BaseModel):
    e: LowerEmailStr

In [14]:
Hello(e="")

ValidationError: 1 validation error for Hello
e
  value is not a valid email address: An email address must have an @-sign. [type=value_error, input_value='', input_type=str]

In [ ]:
class Comp(BaseModel):
    c1: int
class DataOrder(BaseModel):
    c: list[Comp]

In [ ]:
d1 = DataOrder(c=[Comp(c1=1)])
d2 = {"c": [{"c1": 23}]}

In [ ]:
t1 = DataOrder.model_construct(**d1.model_dump())
t1

In [ ]:
t2 = DataOrder.model_construct(**d2)
t2

In [ ]:
from typing import Annotated
from decimal import Decimal
NonNegativeDecimal = Annotated[Decimal, Field(ge=0, decimal_places=2, allow_inf_nan=False)]

In [ ]:
class Cost(BaseModel):
    value: NonNegativeDecimal

In [ ]:
Cost(value=23.98)

In [ ]:
import math
def round_to_base(num: Decimal | float, base: int) -> int:
    """
    Returns a number ceil-rounded to a multiple of provided base

    :param num: the number to round
    :param base: the base to round to
    :return: the rounded number

    >>> round_to_base(61, 4)
    64
    >>> round_to_base(3.5, 1)
    4
    >>> round_to_base(3, 10)
    10
    """
    assert base > 0
    return base * math.ceil(num / base)

In [ ]:
round_to_base(1, 0.01)

In [ ]:
!pip install -U pydantic

In [ ]:
from pydantic import BaseModel, AfterValidator
from typing import Annotated
import json


class User(BaseModel):
    name: str
    id: int

class Test(BaseModel):
    user: dict[str, Annotated[User, AfterValidator(lambda m: m.model_dump())]]


u = User(id=1, name="Y")
t = Test(user={"1": u})
d = {"test": t.user}
json.dumps(d)

'{"test": {"1": {"name": "Y", "id": 1}}}'

In [15]:
from pydantic import AfterValidator, StringConstraints

UpperStr = Annotated[str, AfterValidator(lambda v: v.upper())]
UpperStr2 = Annotated[str, StringConstraints(to_upper=True)]
class TestClass(BaseModel):
    lst: list[UpperStr] | None = None
    lst2: list[UpperStr2] | None = None

TestClass(lst=["a", "b"], lst2=["c", "d"]).model_dump()

{'lst': ['A', 'B'], 'lst2': ['C', 'D']}

In [1]:
from pydantic import BaseModel

class User(BaseModel):
    name: str = "John Doe"  # Explicit default value
    age: int = 30

user = User()  # No arguments provided, uses default values
print(user.name)  # Output: John Doe
print(user.age)   # Output: 30
print(user)       # Output: name='John Doe' age=30

John Doe
30
name='John Doe' age=30


In [8]:
from typing import Optional, List, Any
from pydantic import BaseModel, ConfigDict

class ComputeInstanceOut(BaseModel):
    instance_id: int
    teams: Optional[List[Any]] = None

    model_config = ConfigDict(from_attributes=True)

class CI:
    def __init__(self, id):
        self.instance_id = id
        self.teams = []

c = CI(1)
ComputeInstanceOut.model_validate(c)

ComputeInstanceOut(instance_id=1, teams=[])

In [16]:
# ✅ ПРАЦЮЮЧИЙ ПРИКЛАД: async SQLAlchemy + Pydantic v2 + many-to-many (User ↔ Team)
import asyncio
from typing import List

from sqlalchemy import ForeignKey, Table, Column, Integer, String, select
from sqlalchemy.ext.asyncio import create_async_engine, async_sessionmaker, AsyncSession
from sqlalchemy.orm import DeclarativeBase, Mapped, mapped_column, relationship, selectinload

from pydantic import BaseModel, ConfigDict, Field, field_validator


# ---------- SQLAlchemy models ----------
class Base(DeclarativeBase):
    pass

# Association table з FK
user_teams = Table(
    "user_teams",
    Base.metadata,
    Column("user_id", ForeignKey("users.id"), primary_key=True),
    Column("team_id", ForeignKey("teams.id"), primary_key=True),
)

class Team(Base):
    __tablename__ = "teams"
    id: Mapped[int] = mapped_column(Integer, primary_key=True)
    name: Mapped[str] = mapped_column(String, nullable=False)

    users: Mapped[List["User"]] = relationship(
        secondary=user_teams,
        back_populates="teams",
        lazy="raise",  # 👈 не дозволяємо неявний lazy в async — краще впасти, ніж ловити MissingGreenlet
    )

class User(Base):
    __tablename__ = "users"
    id: Mapped[int] = mapped_column(Integer, primary_key=True)
    name: Mapped[str] = mapped_column(String, nullable=False)

    teams: Mapped[List[Team]] = relationship(
        secondary=user_teams,
        back_populates="users",
    )


# ---------- Pydantic schemas ----------
class TeamSchema(BaseModel):
    id: int
    name: str
    model_config = ConfigDict(from_attributes=True)

class UserSchema(BaseModel):
    id: int
    name: str
    # завжди повертай список (навіть якщо немає зв’язків)
    teams: List[TeamSchema] = Field(default_factory=list)

    # якщо раптом прийде None (залежно від твоєї логіки) — перетвори на []
    @field_validator("teams", mode="before")
    @classmethod
    def none_to_empty_list(cls, v):
        return [] if v is None else v

    model_config = ConfigDict(from_attributes=True)


# ---------- Async demo ----------
async def main():
    engine = create_async_engine("sqlite+aiosqlite:///:memory:", echo=False)
    Session = async_sessionmaker(engine, expire_on_commit=False)

    async with engine.begin() as conn:
        await conn.run_sync(Base.metadata.create_all)

    async with Session() as session:
        # seed
        devops = Team(name="DevOps")
        ml = Team(name="ML")
        user = User(name="Yaryna", teams=[devops])  # одна команда є, інша ні
        session.add_all([user, ml])
        await session.commit()

    # --- 1) Без selectinload: спроба звернутись до .teams викличе помилку (бо lazy="raise")
    async with Session() as session:
        u = await session.scalar(select(User))  # teams НЕ завантажено
        try:
            _ = UserSchema.model_validate(u)  # тут Pydantic торкнеться .teams -> впаде (і це добре)
        except Exception as e:
            print("Без selectinload очікувано впало:", type(e).__name__, "-", e)


# У Jupyter / IPython:
await main()

Без selectinload очікувано впало: ValidationError - 1 validation error for UserSchema
teams
  Error extracting attribute: MissingGreenlet: greenlet_spawn has not been called; can't call await_only() here. Was IO attempted in an unexpected place? (Background on this error at: https://sqlalche.me/e/20/xd2s) [type=get_attribute_error, input_value=<__main__.User object at 0x7d0ee619f8d0>, input_type=User]
    For further information visit https://errors.pydantic.dev/2.10/v/get_attribute_error


In [11]:
!pip install aiosqlite